In [1]:
import pandas as pd
import dask.dataframe as dd

import pickle

In [2]:
class model():
    
    def __init__(self, model, f_const=None):
        self.model = model
        self.f_const = f_const
        self.test_data_filename = 'data_test.csv'
        
    def data_join(self, filename):
    
        # прочтём данные для обучения
        data = pd.read_csv(filename)
        # прочтём данные характеризующие профиль потребления
        features = dd.read_csv('features.csv', sep='\t')
        
        # сформируем набор данных для обучения модели
        ddf = features.merge(data, suffixes=('_features', ''), how='inner', on=['id'])
        df = pd.DataFrame()
        df = ddf.compute()
        # создадим столбец с модулем разности времён (для определения ближайшего профиля)
        df['buy_time_diff'] = df['buy_time_features'] - df['buy_time']
        df['buy_time_diff'] = df['buy_time_diff'].abs()
        # сгруппируем по записям из таблицы data
        df_grouped = df.groupby(by=['id', 'vas_id', 'buy_time'])
        # найдём минимальные разности времён
        min_diffs = df_grouped['buy_time_diff'].min()
        # восстановим датафрейм
        min_diffs = min_diffs.reset_index(level=['id', 'vas_id', 'buy_time'])
        # выберем те профили, которые ближе к дате взаимодействия с услугой
        df = df.merge(min_diffs, how='inner', on=['id', 'vas_id', 'buy_time', 'buy_time_diff'])
        # объединим
        df_grouped = df.groupby(by=['id', 'vas_id', 'buy_time', 'buy_time_diff'])
        # выберем профиль предшествующий взаимодействию с услугой, если их несколько
        min_diffs = df_grouped['buy_time_features'].min()
        # восстановим датафрейм
        min_diffs = min_diffs.reset_index(level=['id', 'vas_id', 'buy_time', 'buy_time_diff'])
        # получим итоговую таблицу
        df = df.merge(min_diffs, how='inner', on=['id', 'vas_id', 'buy_time', 'buy_time_diff', 'buy_time_features'])
        # удалим столбец, который создавали для агрегации
        df.drop('buy_time_diff', axis=1, inplace=True)
        # удалим ненужные столбцы
        df.drop(["Unnamed: 0_features", "Unnamed: 0", "buy_time_features", "buy_time", "id"], axis=1, inplace=True)
        
        return data, df
        
    def fit(self):
        
        data, df_train = self.data_join('data_train.csv')
        
        # перемешаем данные
        df_train = df_train.sample(frac=1).reset_index(drop=True)
        # отделим целевой признак
        train_X_df = df_train.copy()
        train_Y_sr = train_X_df["target"]
        train_X_df.drop(["target"], axis=1, inplace=True)
        # преобразуем vas_id в int
        train_X_df['vas_id'] = train_X_df['vas_id'].astype(int)
        # удалим константные признаки
        train_X_df.drop(self.f_const, axis=1, inplace=True)
        
        # обучим модель
        self.model.fit(train_X_df, train_Y_sr)
        
        return
    
    def predict_proba(self):
        
        data, df_test = self.data_join(self.test_data_filename)
        
        # преобразуем vas_id в int
        df_test['vas_id'] = df_test['vas_id'].astype(int)
        # удалим константные признаки
        df_test.drop(self.f_const, axis=1, inplace=True)
        
        # сделаем предсказание
        preds = self.model.predict_proba(df_test)[:, 1]
        
        data['target'] = preds
        
        data.to_csv("answers_test.csv", index=False)
        
        return 

In [3]:
# теперь загрузим модель
loaded_model = pickle.load(open('model.sav', 'rb'))

In [4]:
%%time
# и сделаем предсказание
loaded_model.predict_proba()

Wall time: 1min 10s


In [5]:
# проверим файл с предсказаниями
answers = pd.read_csv('answers_test.csv')
answers.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,3130519,2.0,1548018000,0.234294
1,1,2000860,4.0,1548018000,0.186640
2,2,1099444,2.0,1546808400,0.188620
3,3,1343255,5.0,1547413200,0.189372
4,4,1277040,2.0,1546808400,0.170746
